## dbt Cloud API Example for Rotating Service Tokens and Storing Them in AWS Secrets Manager
- This script demonstrates how to force a rotation of the dbt Cloud service token by deleting the existing token, creating a new one, and storing the new token details in AWS Secrets Manager.
- This can be achieved via scripting with the dbt Cloud APIs or via the [dbt Cloud Terraform Provider](https://registry.terraform.io/providers/dbt-labs/dbtcloud/latest)

### The following dbt Cloud APIs are used during this script
- [Create Service Token API Call](https://docs.getdbt.com/dbt-cloud/api-v3#/operations/Create%20Service%20Token)
- [Destroy Service Token API Call](https://docs.getdbt.com/dbt-cloud/api-v3#/operations/Destroy%20Service%20Token)

### AWS Service Used
- [AWS Secrets Manager](https://aws.amazon.com/secrets-manager/)


#### If you want to watch an overview Loom video on how this script works:
- [Programmatically Rotating dbt Cloud Service Tokens with AWS Secrets Manager](https://www.loom.com/share/91372b7b419b4a04b69fd406af2914da?sid=707bd4d5-da6b-4a96-9688-bc976f4a828e)

___

In [16]:
import boto3
import json
import requests
from botocore.exceptions import ClientError
import os
from datetime import datetime, timezone

___

#### __Step 1:__ Functions for use in managing AWS Secrets Manager

In [4]:
# ----------------------------------------------------------------
# function to retrieve the secret from AWS Secrets Manager
# ----------------------------------------------------------------
def get_secret(secret_name, region_name):
    """
    Retrieve a secret's value from AWS Secrets Manager.
    Returns the parsed JSON (if applicable) or raw string.
    """
    client = boto3.client('secretsmanager', region_name=region_name)
    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        print(f"Error retrieving secret {secret_name}: {e}")
        return None
    else:
        if 'SecretString' in get_secret_value_response:
            secret_string = get_secret_value_response['SecretString']
            try:
                secret_dict = json.loads(secret_string)
                return secret_dict
            except json.JSONDecodeError:
                return secret_string
        else:
            return get_secret_value_response['SecretBinary']
        
# ----------------------------------------------------------------
# function to update the secret in AWS Secrets Manager
# ----------------------------------------------------------------

def update_aws_secret(secret_name, secret_value, region_name):
    """
    Update an existing AWS Secrets Manager secret with new secret_value.
    secret_value should be a dictionary (it will be JSON-dumped).
    """
    client = boto3.client('secretsmanager', region_name=region_name)
    try:
        response = client.put_secret_value(
            SecretId=secret_name,
            SecretString=json.dumps(secret_value)
        )
        print(f"Secret {secret_name} updated successfully in AWS Secrets Manager.")
        return response
    except ClientError as e:
        print(f"Error updating secret {secret_name}: {e}")
        raise

#### __Step 2:__ dbt Cloud Functions for Handling Service Tokens

In [ ]:
# ----------------------------------------------------------------
# function to delete a dbt Cloud service token
# ----------------------------------------------------------------
def delete_dbt_cloud_service_token(account_id, service_token_id_to_delete, new_dbt_cloud_token, dbt_cloud_access_url='cloud.getdbt.com'):
    """
    Deletes an existing dbt Cloud service token.
    """
    url = f"https://{dbt_cloud_access_url}/api/v3/accounts/{account_id}/service-tokens/{service_token_id_to_delete}/"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {new_dbt_cloud_token}"
    }
    try:
        response = requests.delete(url, headers=headers)
        response.raise_for_status()
        print(f"Deleted dbt Cloud service token id {service_token_id_to_delete}.")
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error deleting dbt Cloud service token id {service_token_id_to_delete}: {e}")
        raise

# ----------------------------------------------------------------
# function to create a new dbt Cloud service token
# ----------------------------------------------------------------
def create_dbt_cloud_service_token(account_id, token_name, dbt_cloud_token, dbt_cloud_access_url='cloud.getdbt.com', 
                                   permission_grants=[
                                        {
                                            "permission_set": "account_admin",
                                            "project_id": None,
                                            "writable_environment_categories": []
                                        }
                                    ]):
    """
    Creates a new dbt Cloud service token.
    The payload might include additional parameters (e.g., scopes) as required.
    """
    url = f"https://{dbt_cloud_access_url}/api/v3/accounts/{account_id}/service-tokens/"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Token {dbt_cloud_token}"
    }
    # Payload can be expanded with additional properties as needed.
    payload = {
        "name": token_name,
        "permission_grants": permission_grants
    }
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        token_data = response.json()['data']
        print(f"Created new dbt Cloud service token with the id: {token_data.get('id')}")
        return token_data
    except requests.exceptions.RequestException as e:
        print(f"Error creating dbt Cloud service token: {e}")
        raise

# ---------------------------------------------------------------------------------------------------------------------------
# [[OPTIONAL]] - function to list all dbt Cloud service tokens
# This function is useful for debugging and ensuring that the token creation and deletion processes are working as expected.
# ---------------------------------------------------------------------------------------------------------------------------
def list_dbt_cloud_service_tokens(account_id, dbt_cloud_token, dbt_cloud_access_url='cloud.getdbt.com'):
    """
    Lists all service tokens for the specified dbt Cloud account.
    """
    url = f"https://{dbt_cloud_access_url}/api/v3/accounts/{account_id}/service-tokens/"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Token {dbt_cloud_token}"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()['data']
    except requests.exceptions.RequestException as e:
        print(f"Error listing service tokens: {e}")
        raise

# ------------------------------------------------------------------------------------------------------------------------------------------
# [[OPTIONAL]] - function to check and make sure there is only one dbt Cloud service token with the name 'dbt_cloud_admin_service_token'
# This function is useful for debugging and ensuring that the token creation and deletion processes are working as expected.
# ------------------------------------------------------------------------------------------------------------------------------------------
def confirm_no_duplicate_admin_tokens(account_id, new_token_id, dbt_cloud_token, token_name='dbt_cloud_admin_service_token', dbt_cloud_access_url='cloud.getdbt.com'):
    """
    Checks for any service tokens with the given name that are not the newly created token and deletes them.
    """
    tokens = list_dbt_cloud_service_tokens(account_id, dbt_cloud_token, dbt_cloud_access_url)
    for token in tokens:
        # Check if the token name matches and is not the new token ID and is active (state == 1)
        # dbt Cloud will list metadata about old tokens via the API for audit purposes
        if token.get('name') == token_name and token.get('id') != new_token_id and token.get('state') == 1:
            print(f"Found duplicate token {token.get('id')} with name {token_name}. Deleting...")
            delete_dbt_cloud_service_token(account_id, token.get('id'), dbt_cloud_token, dbt_cloud_access_url)


#### __Step 3:__ Putting it together and rotating dbt Cloud service tokens 

In [ ]:
# ------------------------------------------------------------------------------------------------------------------------------------------
# Configuration – adjust these variables as needed.
# ------------------------------------------------------------------------------------------------------------------------------------------
dbt_cloud_account_id = 12345  # your dbt Cloud account id
dbt_cloud_access_url = 'cloud.getdbt.com'  # or your custom URL if needed
aws_region = "us-east-2"  # region for AWS secrets manager
secret_name = "dbt_cloud_admin_service_token"  # AWS secret name

# ------------------------------------------------------------------------------------------------------------------------------------------
# Step 1: get the current dbt Cloud service token from secrets manager
# ------------------------------------------------------------------------------------------------------------------------------------------
secrets_manager_secret = get_secret(secret_name, aws_region)
existing_dbt_cloud_api_token_id = secrets_manager_secret.get("service_token_id") 
existing_dbt_cloud_api_token = secrets_manager_secret.get("service_token")  # dbt Cloud API token

# ------------------------------------------------------------------------------------------------------------------------------------------
# Step 2: Create a new dbt Cloud service token.
# ------------------------------------------------------------------------------------------------------------------------------------------
rotated_dbt_cloud_api_token_info = create_dbt_cloud_service_token(dbt_cloud_account_id, secret_name, existing_dbt_cloud_api_token, dbt_cloud_access_url)
# # grab the new token
new_dbt_cloud_api_token = rotated_dbt_cloud_api_token_info.get("token_string")
new_dbt_cloud_api_token_id = rotated_dbt_cloud_api_token_info.get("id")


# ------------------------------------------------------------------------------------------------------------------------------------------
# Step 3: Update AWS Secrets Manager with the new service token details.
# ------------------------------------------------------------------------------------------------------------------------------------------
# You might store additional metadata if desired.
secret_value = {
    "service_token_id": new_dbt_cloud_api_token_id,
    "service_token": new_dbt_cloud_api_token,
    "last_rotated_datetime_utc": datetime.now(timezone.utc).isoformat()
}
# run the update
update_aws_secret(secret_name, secret_value, aws_region)


# ------------------------------------------------------------------------------------------------------------------------------------------
# Step 4: Delete the old dbt Cloud service token.
# ------------------------------------------------------------------------------------------------------------------------------------------
delete_dbt_cloud_service_token(dbt_cloud_account_id, existing_dbt_cloud_api_token_id, new_dbt_cloud_api_token, dbt_cloud_access_url)

# ------------------------------------------------------------------------------------------------------------------------------------------
# Step 5: [[OPTIONAL]] Verify that only the new admin token exists, if any dupes found under the same name delete them
# ------------------------------------------------------------------------------------------------------------------------------------------
confirm_no_duplicate_admin_tokens(dbt_cloud_account_id, new_dbt_cloud_api_token_id, new_dbt_cloud_api_token, secret_name, dbt_cloud_access_url)

# ------------------------------------------------------------------------------------------------------------------------------------------
# END
# ------------------------------------------------------------------------------------------------------------------------------------------


Created new dbt Cloud service token with the id: 55641
Secret dbt_cloud_admin_service_token_new updated successfully in AWS Secrets Manager.
Deleted dbt Cloud service token id 55640.


# END OF SCRIPT